In [8]:
from chain import get_sentence
import cPickle as pickle
import tweepy
from datetime import date, timedelta
from twitter_keys import keys

In [9]:
with open('Pickles/forward_dict.pkl') as f:
    f_dict = pickle.load(f)

with open('Pickles/reversed_dict.pkl') as f:
    r_dict = pickle.load(f)

In [10]:
CONSUMER_KEY = keys['consumer_key']
CONSUMER_SECRET = keys['consumer_secret']
ACCESS_TOKEN = keys['access_token']
ACCESS_TOKEN_SECRET = keys['access_token_secret']

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

tweet_replies = []

today = date.today().strftime("%Y-%m-%d")
yesterday = date.today() - timedelta(1)

# ******************************************************
# Queries Popular Tweets directed at Candidates or using
# well-known Hashtags and responds to all queried tweets.
# ******************************************************
hashtags = ["#FeelTheBern OR #ImWithHer OR #MakeAmericaGreatAgain OR #Trump2016 \
OR @BernieSanders OR @HillaryClinton OR @realDonaldTrump"]

top_tweets = tweepy.Cursor(api.search, q=hashtags, count=100, result_type='popular', include_entities=True, monitor_rate_limit=True, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, lang="en", since=yesterday, until=today).items()

for status in top_tweets:
    if status.favorite_count > 1000:
        tweet_replies.append(status)

# ***************************************************
# Queries Tweets directed at or in reply to
# FeelTheBern_Bot and responds to all queried tweets.
# ***************************************************
bernbot_query = "to:FeelTheBern_Bot OR @FeelTheBern_Bot"

bernbot_tweets = tweepy.Cursor(api.search, q=bernbot_query, count=100, result_type='recent', include_entities=True, monitor_rate_limit=True, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, lang="en", since=yesterday, until=today).items()

for status in bernbot_tweets:
    tweet_replies.append(status)

# ************************************************
# Queries Tweets from Presidential Candidates and
# responds to those with greater than 500 likes.
# ************************************************
candidate_query = "from:BernieSanders OR from:HillaryClinton OR from:realDonaldTrump"

candidate_tweets = tweepy.Cursor(api.search, q=candidate_query, count=100, result_type='recent', include_entities=True, monitor_rate_limit=True, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, lang="en", since=yesterday, until=today).items()

for status in candidate_tweets:
    if status.favorite_count > 500:
        tweet_replies.append(status)


# DROPS ANY DUPLICATE TWEETS SO IT DOES NOT RESPOND TWICE
tweet_replies = set(tweet_replies)

In [11]:
best_tweets = []
for i, tweet in enumerate(tweet_replies):
    resp_length = 141
    text = tweet.text
    sn = tweet.user.screen_name
    response = ':'

    while resp_length > 140 or ':' in response:
        response = get_sentence(text, f_dict, r_dict, randomness = 1) 
        response = "@{0} {1}".format(sn, response)
        resp_length = len(response)
    
    print i, '----', tweet.id
    print '@', sn, text
    print response
    print '*************'
    best_tweets.append([response, tweet.id])

0 ---- 727540765765828609
@ realDonaldTrump Polls close at 6pm! #INPrimary #Trump2016 #VoteTrump 
https://t.co/DT9WkYAGEK
@realDonaldTrump So don't tell me that we ought to close that Internet thing.
*************
1 ---- 727582702749159424
@ HillaryClinton We don’t have time to waste on Trump's fake solutions that won’t do a thing to help your families.
https://t.co/b4aq8SVVAB
@HillaryClinton One at a time, go ahead and respond.
*************
2 ---- 727563120571494401
@ IngrahamAngle Republicans who prefer @HillaryClinton to @realDonaldTrump shd spare us all the fence-sitting &amp; posturing &amp; just make it official.
@IngrahamAngle As you too know, I would note that the scripture tells us, you shall know them by their fruit.
*************
3 ---- 727319608663875585
@ realDonaldTrump Thank you South Bend, Indiana! Everyone get out &amp; #VoteTrump tomorrow! #INPrimary 
https://t.co/rj11BPzqDI https://t.co/xEBlvGrny1
@realDonaldTrump He doesn't get to yell the whole time.
*************

In [12]:
best_list = [0,1,2,3,6,10,11,18]
for i, tweet in enumerate(best_tweets):
    if i in best_list:
        print i 
        print tweet[0], tweet[1]
        s = api.update_status(tweet[0], tweet[1])

0
@realDonaldTrump So don't tell me that we ought to close that Internet thing. 727540765765828609
1
@HillaryClinton One at a time, go ahead and respond. 727582702749159424
2
@IngrahamAngle As you too know, I would note that the scripture tells us, you shall know them by their fruit. 727563120571494401
3
@realDonaldTrump He doesn't get to yell the whole time. 727319608663875585
6
@HillaryClinton Going to bring jobs back and I'll start bringing them back very fast. 727637840620879873
10
@DanScavino Polls go up. 727322509712875524
11
@BernieSanders Frankly, what I added, which I think was smart power at its best, is that we got to get wages going up again. 727296128031449088
18
@BernieSanders We should tell the Chinese, by the Kurds, the Christians and the moderate Sunnis. 727580488383238144
